# Scraping data from 2022

In [1]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [2]:
dates = ["01.12.2022", "02.12.2022", "03.12.2022", "04.12.2022", "05.12.2022", "06.12.2022", "07.12.2022", "08.12.2022", "09.12.2022", "10.12.2022", "11.12.2022", "12.12.2022", "13.12.2022", "14.12.2022", "15.12.2022", "16.12.2022", "17.12.2022", "18.12.2022", "19.12.2022", "20.12.2022", "21.12.2022", "22.12.2022", "23.12.2022", "24.12.2022", "25.12.2022", "26.12.2022", "27.12.2022", "28.12.2022", "29.12.2022", "30.12.2022", "31.12.2022", "01.01.2023", "02.01.2023", "03.01.2023", "04.01.2023"]

In [3]:
df = pd.read_csv("data/links.csv")

total_df = pd.DataFrame()
df.head(20)

,country,area,link
0,Albania,BZN AL,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAL-KESH-----5!BZN|10YAL-KESH-----5&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
1,Austria,BZN AT,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAT-APG------L!BZN|10YAT-APG------L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
2,Austria,BZN DE-AT-LU,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAT-APG------L!BZN|10Y1001A1001A63L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
3,Belgium,BZN BE,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YBE----------2!BZN|10YBE----------2&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
4,Bosnia and Herz.,BZN BA,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YBA-JPCC-----D!BZN|10YBA-JPCC-----D&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
5,Bulgaria,BZN BG,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YCA-BULGARIA-R!BZN|10YCA-BULGARIA-R&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
6,Croatia,BZN HR,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YHR-HEP------M!BZN|10YHR-HEP------M&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
7,Cyprus,BZN CY,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YCY-1001A0003J!BZN|10YCY-1001A0003J&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
8,Czech Rep.,BZN CZ,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YCZ-CEPS-----N!BZN|10YCZ-CEPS-----N&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
9,Czech Rep.,BZN CZ-DE-SK,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YCZ-CEPS-----N!BZN|10YDOM-CZ-DE-SKK&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)


In [4]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [5]:
for date in dates:
    df = df.replace(to_replace=r'\d\d.\d\d.202\d', value= date, regex=True)

    for index, row in df.iterrows():
        url = row['link']

        country = row['country']
        area = row['area']
    
        await page.goto(url)

        time.sleep(3)
        await page.get_by_text("PT60M").click()

        time.sleep(3)

        tables = pd.read_html(await page.content(), flavor = 'html5lib')
        data = tables[0]
        data.insert(0, 'date', date)
        data.insert(1, 'country', country)
        data.insert(2, 'area', area)
        total_df = pd.concat([total_df, data], axis=0)
    
    print("scraped", date)
    print("scraped", url)
    total_df

scraped 01.12.2022
scraped https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.12.2022+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|17Y0000009369493&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
scraped 02.12.2022
scraped https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=02.12.2022+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|17Y0000009369493&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
scraped 03.12.2022
scraped https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=03.12.2022+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|17Y000

TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for get_by_text("PT60M")
  locator resolved to visible <label for="resolution.values3">PT60M</label>
attempting click action
  waiting for element to be visible, enabled and stable
  element is visible, enabled and stable
  scrolling into view if needed
  done scrolling
  <div class="ui-widget-overlay ui-front"></div> intercepts pointer events
retrying click action, attempt #1
  waiting for element to be visible, enabled and stable
  element is visible, enabled and stable
  scrolling into view if needed
  done scrolling
  <div class="ui-widget-overlay ui-front"></div> intercepts pointer events
retrying click action, attempt #2
  waiting 20ms
  waiting for element to be visible, enabled and stable
  element is visible, enabled and stable
  scrolling into view if needed
  done scrolling
  <div class="ui-widget-overlay ui-front"></div> intercepts pointer events
retrying click actio

In [8]:
total_df.columns = ['date','country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']
total_df.tail(50)

# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
data = total_df[['date', 'country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']]
data.head(50)

In [9]:
total_df.tail(50)

,date,country,area,mtu,price(currency/mwh),price(eur/mwh)
22,14.12.2022,Italy,BZN IT-SACOAC,22:00 - 23:00,NaN,323.99
23,14.12.2022,Italy,BZN IT-SACOAC,23:00 - 00:00,NaN,283.76
0,14.12.2022,Italy,BZN IT-SACODC,00:00 - 01:00,NaN,338.37
1,14.12.2022,Italy,BZN IT-SACODC,01:00 - 02:00,NaN,321.00
2,14.12.2022,Italy,BZN IT-SACODC,02:00 - 03:00,NaN,306.54
3,14.12.2022,Italy,BZN IT-SACODC,03:00 - 04:00,NaN,289.36
4,14.12.2022,Italy,BZN IT-SACODC,04:00 - 05:00,NaN,292.79
5,14.12.2022,Italy,BZN IT-SACODC,05:00 - 06:00,NaN,312.14
6,14.12.2022,Italy,BZN IT-SACODC,06:00 - 07:00,NaN,368.82
7,14.12.2022,Italy,BZN IT-SACODC,07:00 - 08:00,NaN,494.97


In [10]:
data = data.drop(['price(currency/mwh)'], axis=1)
data

,date,country,area,mtu,price(eur/mwh)
0,01.12.2022,Albania,BZN AL,00:00 - 01:00,NaN
1,01.12.2022,Albania,BZN AL,01:00 - 02:00,NaN
2,01.12.2022,Albania,BZN AL,02:00 - 03:00,NaN
3,01.12.2022,Albania,BZN AL,03:00 - 04:00,NaN
4,01.12.2022,Albania,BZN AL,04:00 - 05:00,NaN
...,...,...,...,...,...
19,14.12.2022,Italy,BZN IT-Sardina,19:00 - 20:00,376.82
20,14.12.2022,Italy,BZN IT-Sardina,20:00 - 21:00,352.99
21,14.12.2022,Italy,BZN IT-Sardina,21:00 - 22:00,324.99
22,14.12.2022,Italy,BZN IT-Sardina,22:00 - 23:00,323.99


In [11]:
data.to_csv("data/prices-hourly-111222-201222.csv", mode='a', index=True, header=False)

In [ ]:
df = pd.read_csv("data/prices-hourly-111222-201222.csv")
df['date'] = pd.to_datetime(df['date'],format="%d.%m.%Y")
df = df.sort_values(by=['date', 'country','area','mtu'])
df

In [ ]:
data = df.drop(['Unnamed: 0'], axis=1)
#data = data.drop(['price(currency/mwh)'], axis=1) 
data

In [ ]:
data.to_csv("data/prices-hourly-011222-040123.csv", mode='a', index=False, header=True)

In [ ]:
#data_day = round(data.groupby(['date','country'])['price(eur/mwh)'].mean())
data_day = pd.read_csv("data/prices-hourly-011222-040123.csv")
data_day = round(data.groupby(['date', 'country']).agg({'price(eur/mwh)': 'mean'}))

# Reset the index to get the date on each row.
data_day = data_day.reset_index()
data_day.tail(50)

data_day.to_csv("data/prices-daily-011222-040123.csv", index=False, header=True)